In [ ]:
import pandas as pd
import requests
from io import BytesIO
from difflib import get_close_matches

class SistemaAutorizacionMejorado:
    def __init__(self):
        self.sheets_publicas = {
            'CAPITA_BOGOTA_CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx',
            'CAPITA_BOGOTA_NORTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_BOGOTA_SUR': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx',
            'CAPITA_BOGOTA_SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx',
            'CAPITA_INSUMOS_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx',
            'CAPITA_MEDICAMENTOS_DROGUERIAS_CAFAM': 'https://docs.google.com/spreadsheets/d/17pEopDY47lPmTuELX5MNYI--PgrtPgIR/export?format=xlsx',
            'EXCLUSIONES_MEDICAMENTOS_CAPITA': 'https://docs.google.com/spreadsheets/d/1AaAgFYrTwt2jh8oS_OQ4eaHxMvVg6mrr/export?format=xlsx',
            'MEDICAMENTOS_CAPITA_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx',
            'CAPITA_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx'
        }

        # Columnas objetivo con variaciones posibles
        self.columnas_objetivo = {
            'MEDICAMENTOS': ['medicamento', 'medicamentos', 'nombre medicamento'],
            'DESCRIPCION_INSUMOS': ['descripción insumos', 'descripcion insumos', 'detalle insumos'],
            'CODIGO_CUPS': ['código cups', 'codigo cups', 'cups', 'cod cups'],
            'DESCRIPCION_SERVICIO': ['descripción servicio', 'descripcion servicio', 'detalle servicio'],
            'DESCRIPCION_MEDICAMENTOS': ['descripción medicamento', 'descripcion medicamento', 'detalle medicamento']
        }

        self.tablas = {}
        self.cargar_tablas()

    def encontrar_columna(self, df):
        """Encuentra las columnas objetivo con coincidencia flexible"""
        columnas_encontradas = {}

        for col in df.columns:
            col_lower = str(col).lower().strip()

            # Buscar coincidencias para cada tipo de columna
            for tipo, variantes in self.columnas_objetivo.items():
                for variante in variantes:
                    if variante in col_lower:
                        columnas_encontradas[tipo] = col
                        break

        return columnas_encontradas

    def cargar_tablas(self):
        """Carga las tablas con detección mejorada de columnas"""
        print("Cargando tablas con búsqueda flexible de columnas...")

        for nombre, url in self.sheets_publicas.items():
            try:
                response = requests.get(url, timeout=20)
                excel_file = BytesIO(response.content)
                xls = pd.ExcelFile(excel_file)

                hojas_validas = {}
                for hoja in xls.sheet_names:
                    df = pd.read_excel(excel_file, sheet_name=hoja)

                    # Encontrar columnas objetivo
                    columnas = self.encontrar_columna(df)

                    if columnas:
                        # Crear nuevo DataFrame solo con las columnas encontradas
                        df_filtrado = df[list(columnas.values())].copy()
                        df_filtrado.columns = columnas.keys()  # Estandarizar nombres
                        df_filtrado = df_filtrado.dropna(how='all')

                        if not df_filtrado.empty:
                            hojas_validas[hoja] = df_filtrado

                if hojas_validas:
                    self.tablas[nombre] = hojas_validas
                    print(f"\u2713 {nombre}: Columnas encontradas - {list(next(iter(hojas_validas.values())).columns)}")
                else:
                    print(f"\u26A0 {nombre}: No se detectaron las columnas objetivo. Revisar nombres.")
                    print(f"   Columnas disponibles en la primera hoja: {list(pd.read_excel(excel_file, sheet_name=xls.sheet_names[0]).columns)}")

            except Exception as e:
                print(f"\u274C Error cargando {nombre}: {str(e)}")

    def buscar_servicio(self, ciudad, ips, servicio):
        """Busca en todas las tablas cargadas según ciudad e IPS"""
        ciudad = ciudad.upper()
        ips = ips.upper()
        servicio = str(servicio).strip().lower()
        resultados = []

        # Determinar tablas relevantes según ciudad e IPS
        tablas_relevantes = []
        if 'BOGOT' in ciudad:
            if 'COLSUBSIDIO' in ips:
                tablas_relevantes.extend(['CAPITA_CAFAM_COLSUBSIDIO', 'MEDICAMENTOS_CAPITA_COLSUBSIDIO'])
            if 'NORTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_NORTE')
            if 'CENTRO' in ips or 'ORIENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_CENTRO_ORIENTE')
            if 'SUROCCIDENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_SUROCCIDENTE')
        elif 'SOACHA' in ciudad and 'CAFAM' in ips:
            tablas_relevantes.extend(['CAPITA_MEDICAMENTOS_DROGUERIAS_CAFAM', 'CAPITA_INSUMOS_CAFAM_COLSUBSIDIO'])

        # Buscar en las tablas relevantes
        for tabla_nombre in tablas_relevantes:
            if tabla_nombre in self.tablas:
                for hoja, df in self.tablas[tabla_nombre].items():
                    for col in df.columns:
                        try:
                            mask = df[col].astype(str).str.lower().str.contains(servicio)
                            if mask.any():
                                resultados.append({
                                    'encontrado': True,
                                    'datos': df[mask].iloc[0].to_dict(),
                                    'tabla': tabla_nombre,
                                    'hoja': hoja,
                                    'tipo': 'CAPITA' if 'CAPITA' in tabla_nombre else 'PGP'
                                })
                                return resultados[0]  # Devolver el primer resultado encontrado
                        except:
                            continue

        return {'encontrado': False}

    def iniciar_proceso(self):
        """Flujo interactivo del sistema con los 5 pasos"""
        print("\n=== SISTEMA DE AUTORIZACIÓN - VALIDACIÓN EN 5 PASOS ===")

        # Paso 1-3: Ciudad, IPS, Servicio
        ciudad = input("\n1. Ciudad de residencia (Bogotá/Soacha): ").strip()
        ips = input("2. IPS primaria (ej: Colsubsidio, Subred Norte): ").strip()
        servicio = input("3. Código CUPS o nombre del servicio: ").strip()

        # Buscar en tablas
        resultado = self.buscar_servicio(ciudad, ips, servicio)

        if resultado.get('encontrado', False):
            print("\n=== RESULTADO ===")
            print(f"\u2713 **Servicio encontrado en:** {resultado['tabla']} (Hoja: {resultado['hoja']})")
            print(f"\U0001F539 **Tipo:** {resultado['tipo']}")
            print("\n**Detalles del servicio:**")
            for k, v in resultado['datos'].items():
                print(f"- {k}: {v}")
        else:
            # Paso 4-5: Si no se encuentra, pedir más datos
            print("\n=== RESULTADO ===")
            print("\U0001F534 Servicio NO encontrado en tablas de prestación directa.")
            ips_remite = input("\n4. IPS que remite la orden: ").strip()
            diagnostico = input("5. Diagnóstico de la orden médica: ").strip()

            print("\n**Acciones requeridas:**")
            print(f"- Validar documentación con IPS {ips_remite}")
            print(f"- Escalar al comité de autorizaciones (Diagnóstico: {diagnostico})")

if __name__ == "__main__":
    sistema = SistemaAutorizacionMejorado()
    sistema.iniciar_proceso()